# Opioid Misuse Prediction Models

*Yiyu Wang 2024/04/16*

In [ ]:
data_dir = '../data/'
figures_dir = '../figures/'
model_dir = '../model_results/'

import joblib
import numpy as np
import glob
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge, ElasticNet
import warnings

warnings.filterwarnings('ignore')



In [ ]:
PREDICTOR_COLUMNS=['k23_age', 'demo_gender', 'demo_hispanic', 'demo_ethnicity',
       'demo_income', 'demo_education', 'demo_legal', 'demo_employment___1',
       'demo_employment___2', 'demo_employment___3', 'demo_employment___4',
       'demo_employment___5', 'demo_employment___6', 'demo_employment___7',
       'demo_employment___8', 'demo_employment___9', 'demo_employment___99',
       'demo_disability', 'demo_marital', 'mh_accident', 'mh_pain_duration',
       'promis_pi_01', 'promis_pi_02', 'promis_pi_03', 'opioid_years_v2',
       'meds_more_v2', 'PainInT', 'AngerT', 'AnxietyT', 'DepressT', 'FatigueT',
       'GlobalpT', 'GlobalmT', 'PhyFxT', 'SleepDisT', 'audittot', 'AUDITpos',
       'pcstotal', 'pcs_help', 'pcs_rum', 'pcs_mag', 'dasttot', 'c_eactotl',
       'aeqtot', 'ctq_emo_abu', 'ctq_phy_abu', 'ctq_emo_neg', 'ctq_phy_neg',
       'ctq_sex_abu', 'ctqtot', 'mh_psychological_yes_binary']
print(PREDICTOR_COLUMNS)
print('n predictors =', len(PREDICTOR_COLUMNS))
SEED = 2025

def split_data(df, split_data = 'split', outcome = 'ouddx', predictor_columns = PREDICTOR_COLUMNS, test_size = 0.2, SEED = SEED):
    if split_data == 'cohort':

        y_train, y_test, y_val = df.loc[df['cohort'] == 'train', outcome], df.loc[df['cohort'] == 'test', outcome], df.loc[df['cohort'] == 'val', outcome]
        X_train, X_test, X_val = df.loc[df['cohort'] == 'train', predictor_columns], df.loc[df['cohort'] == 'test', predictor_columns], df.loc[df['cohort'] == 'validation', predictor_columns]

        print(X_train.shape, X_test.shape, X_val.shape)
    elif split_data == 'split':
        X = df[predictor_columns]
        y = df[outcome]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=SEED)
        
    else:
        X_train, X_test, y_train, y_test = [], [], [], []
        print('split_data must be either "split" or "cohort"')
    return X_train, X_test, y_train, y_test
    # return {'X_train': X_train,'X_test':X_test, 'y_train': y_train, 'y_test':y_test}



In [ ]:
df = pd.read_csv(data_dir + 'M_K23_ML_reduced_imputed.csv')
df.head()

## Permutation for the best model

In [ ]:
model_results = joblib.load(model_dir + 'rank_model_results.pkl')
# Extract the r_scores for each model
r_scores = [result['r_scores'] for result in model_results.values()]
max_mean_index = np.argmax([np.mean(scores) for scores in r_scores])

In [ ]:
from sklearn.inspection import permutation_importance
final_best_model_name = 'ElasticNet'

# load
model = joblib.load(model_dir + f'{final_best_model_name}_rank_best_model.pkl')


X_train, X_test, y_train, y_test = split_data(df, split_data = 'split', outcome = 'commtot', predictor_columns=PREDICTOR_COLUMNS, test_size = 0.3, SEED = SEED)
model.fit(X_train, y_train)


In [ ]:
perm_importance = permutation_importance(model, X_test, y_test, n_repeats=30, random_state=SEED)
max_perm_importance = np.argmax(perm_importance.importances_mean)
print(PREDICTOR_COLUMNS[max_perm_importance])

In [ ]:
from scipy.stats import pearsonr

# Predict using the model
y_pred = model.predict(X_test)

# Calculate the Pearson correlation coefficient and p-value
r, p_value = pearsonr(y_test, y_pred)

print(f"Pearson correlation coefficient: {r}")
print(f"P-value: {p_value}")


In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import ElasticNet
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

# Predict using the model
y_pred = model.predict(X_test)

# Calculate the Pearson correlation coefficient for the original data
r_original = np.median(r_scores[max_mean_index])

# Perform permutation testing
n_permutations = 100
count = 0
permuted_rs = []

for _ in range(n_permutations):
    # Permute the response variable
    X_test_permuted = np.random.permutation(X_test)
    y_pred_permuted = model.predict(X_test_permuted)
    
    # Calculate the Pearson correlation coefficient for the permuted data
    r_permuted, _ = pearsonr(y_pred_permuted, y_pred)
    permuted_rs.append(r_permuted)
    
    # Count how many permuted r's are greater than or equal to the observed r
    if abs(r_permuted) >= abs(r_original):
        count += 1

# Calculate the p-value
p_value = count / n_permutations

print(f"Original Pearson correlation coefficient: {r_original}")
print(f"P-value from permutation test: {p_value}")

# Plot the distribution of permuted correlation coefficients
plt.hist(permuted_rs, bins=30, color='gray', edgecolor='black', alpha=0.7)
plt.axvline(x=r_original, color='red', linestyle='dashed', linewidth=1)
plt.title('Distribution of Permuted Correlation Coefficients')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import ElasticNet
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

# Predict using the model
y_pred = model.predict(X_test)

# Calculate the Pearson correlation coefficient for the original data
r_original = np.mean(r_scores[max_mean_index])

# Perform permutation testing
n_permutations = 100
count = 0
permuted_rs = []

for _ in range(n_permutations):
    # Permute the response variable
    X_test_permuted = np.random.permutation(X_test)
    y_pred_permuted = model.predict(X_test_permuted)
    
    # Calculate the Pearson correlation coefficient for the permuted data
    r_permuted, _ = pearsonr(y_pred_permuted, y_pred)
    permuted_rs.append(r_permuted)
    
    # Count how many permuted r's are greater than or equal to the observed r
    if abs(r_permuted) >= abs(r_original):
        count += 1

# Calculate the p-value
p_value = count / n_permutations

print(f"Original Pearson correlation coefficient: {r_original}")
print(f"P-value from permutation test: {p_value}")

# Plot the distribution of permuted correlation coefficients
plt.hist(permuted_rs, bins=30, color='gray', edgecolor='black', alpha=0.7)
plt.axvline(x=r_original, color='red', linestyle='dashed', linewidth=1)
plt.title('Distribution of Permuted Correlation Coefficients')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Frequency')
plt.show()


## Feature Importance using ablation analysis:

Ablation analysis: removing a feature ("ablation") in the model prediction shows how much the removal impacts the model performance. The larger the reduction, the greater impact this feature has on the model prediction. 

In [ ]:

# load ablation
ablation_results = joblib.load( model_dir + f'{final_best_model_name}_ablation_results.pkl')

   

In [ ]:
ablated_r_scores = [result['r_scores'] for result in ablation_results.values()]
# Permutation testing
n_permutations = 1000
predictor_delta_p_value = []

predictors_of_interest = ['AngerT', 'aeqtot', 'pcstotal', 'FatigueT', 'PainInT']
fig, axs = plt.subplots(len(predictors_of_interest), 1, figsize=(8, 6*len(predictors_of_interest)))

for i, c in enumerate(predictors_of_interest):
    ablated_r_perm_list, delta_r_perm_list = [], []
    count = 0

    delta_r = r_original - ablation_results[c]['median_r']
    column_index = PREDICTOR_COLUMNS.index(c)

    for seed in range(n_permutations):
        
        X_test_permuted = np.random.permutation(X_test)
        y_pred_permuted = model.predict(X_test_permuted)
        
        X_test_ablated_permuted = X_test_permuted.copy()
        X_test_ablated_permuted[:,column_index] = 0

        y_pred_ablated_permuted = model.predict(X_test_ablated_permuted)

        ablated_r_perm = np.corrcoef(y_pred_ablated_permuted, y_test)[0, 1]
        r_perm = np.corrcoef(y_pred_permuted, y_test)[0, 1]
        delta_r_perm = r_perm - ablated_r_perm

        ablated_r_perm_list.append(ablated_r_perm)
        delta_r_perm_list.append(delta_r_perm)

        if abs(delta_r_perm) >= abs(delta_r):
            count += 1
    print('95% percentile value', np.percentile(delta_r_perm_list, 95))
    p_value = count / n_permutations
    predictor_delta_p_value.append(p_value)
    print(f"Ablated Predictor: {c}")
    print(f"Reduction in r after ablation: {delta_r}")
    print(f"P-value: {p_value}")

    # two sample t test: whether two distributions are significantly different
    from scipy.stats import ttest_ind
    t_stat, p_value = ttest_ind(r_original-ablation_results[c]['r_scores'], delta_r_perm_list)
    print(f"t-statistic: {t_stat}, p-value: {p_value}")

    # Plot the distribution of permuted correlation coefficients
    axs[i].hist(delta_r_perm_list, bins=50, color='gray', edgecolor='black', alpha=0.7)
    axs[i].hist(r_original-ablation_results[c]['r_scores'], bins=50, color='blue', edgecolor='black', alpha=0.7)
    axs[i].axvline(x=delta_r, color='red', linestyle='dashed', linewidth=1)
    axs[i].set_title(f'Distribution of Permuted Correlation Coefficients for {c}')
    axs[i].set_xlabel('Correlation Coefficient')
    axs[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()
    


In [ ]:
ablated_r_scores = [result['r_scores'] for result in ablation_results.values()]
# Permutation testing
n_permutations = 100
predictor_delta_p_value = []

predictors_of_interest = ['AngerT', 'aeqtot', 'pcstotal', 'FatigueT', 'PainInT']
fig, axs = plt.subplots(len(predictors_of_interest), 1, figsize=(8, 6*len(predictors_of_interest)))

for i, c in enumerate(predictors_of_interest):
    ablated_r_perm_list, delta_r_perm_list = [], []
    count = 0

    delta_r = r_original - ablation_results[c]['mean_r']
    column_index = PREDICTOR_COLUMNS.index(c)

    for seed in range(n_permutations):
        
        X_test_permuted = np.random.permutation(X_test)
        y_pred_permuted = model.predict(X_test_permuted)
        
        X_test_ablated_permuted = X_test_permuted.copy()
        X_test_ablated_permuted[:,column_index] = 0

        y_pred_ablated_permuted = model.predict(X_test_ablated_permuted)

        ablated_r_perm = np.corrcoef(y_pred_ablated_permuted, y_test)[0, 1]
        r_perm = np.corrcoef(y_pred_permuted, y_test)[0, 1]
        delta_r_perm = r_perm - ablated_r_perm

        ablated_r_perm_list.append(ablated_r_perm)
        delta_r_perm_list.append(delta_r_perm)

        if abs(delta_r_perm) >= abs(delta_r):
            count += 1
    print('95% percentile value', np.percentile(delta_r_perm_list, 95))
    p_value = count / n_permutations
    predictor_delta_p_value.append(p_value)
    print(f"Ablated Predictor: {c}")
    print(f"Reduction in r after ablation: {delta_r}")
    print(f"P-value: {p_value}")

    # two sample t test: whether two distributions are significantly different
    from scipy.stats import ttest_ind
    t_stat, p_value = ttest_ind(r_original-ablation_results[c]['r_scores'], delta_r_perm_list)
    print(f"t-statistic: {t_stat}, p-value: {p_value}")

    # Plot the distribution of permuted correlation coefficients
    axs[i].hist(delta_r_perm_list, bins=50, color='gray', edgecolor='black', alpha=0.7)
    axs[i].hist(r_original-ablation_results[c]['r_scores'], bins=50, color='blue', edgecolor='black', alpha=0.7)
    axs[i].axvline(x=delta_r, color='red', linestyle='dashed', linewidth=1)
    axs[i].set_title(f'Distribution of Permuted Correlation Coefficients for {c}')
    axs[i].set_xlabel('Correlation Coefficient')
    axs[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()
    


### AngerT, aeqtot, and pcstotal are the top three features that had the most impact to the model predictions

In [ ]:
import shap

PREDICTOR_COLUMNS=['k23_age', 'demo_gender', 'demo_hispanic', 'demo_ethnicity',
       'demo_income', 'demo_education', 'demo_legal', 'demo_employment___1',
       'demo_employment___2', 'demo_employment___3', 'demo_employment___4',
       'demo_employment___5', 'demo_employment___6', 'demo_employment___7',
       'demo_employment___8', 'demo_employment___9', 'demo_employment___99',
       'demo_disability', 'demo_marital', 'mh_accident', 'mh_pain_duration',
       'promis_pi_01', 'promis_pi_02', 'promis_pi_03', 'opioid_years_v2',
       'meds_more_v2', 'PainInT', 'AngerT', 'AnxietyT', 'DepressT', 'FatigueT',
       'GlobalpT', 'GlobalmT', 'PhyFxT', 'SleepDisT', 'audittot', 'AUDITpos',
       'pcstotal', 'pcs_help', 'pcs_rum', 'pcs_mag', 'dasttot', 'c_eactotl',
       'aeqtot', 'ctq_emo_abu', 'ctq_phy_abu', 'ctq_emo_neg', 'ctq_phy_neg',
       'ctq_sex_abu', 'ctqtot', 'mh_psychological_yes_binary']
print(PREDICTOR_COLUMNS)
print('n predictors =', len(PREDICTOR_COLUMNS))
SEED = 2024

def split_data(df, split_data = 'split', outcome = 'ouddx', predictor_columns = PREDICTOR_COLUMNS, test_size = 0.2, SEED = SEED):
    from sklearn.model_selection import train_test_split
    if split_data == 'cohort':
        y_train, y_test, y_val = df.loc[df['cohort'] == 'train', outcome], df.loc[df['cohort'] == 'test', outcome], df.loc[df['cohort'] == 'val', outcome]
        X_train, X_test, X_val = df.loc[df['cohort'] == 'train', predictor_columns], df.loc[df['cohort'] == 'test', predictor_columns], df.loc[df['cohort'] == 'validation', predictor_columns]
    elif split_data == 'split':
        X = df[predictor_columns]
        y = df[outcome]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=SEED) 
    else:
        X_train, X_test, y_train, y_test = [], [], [], []
        print('split_data must be either "split" or "cohort"')
    return X_train, X_test, y_train, y_test



df = pd.read_csv(data_dir + 'M_K23_ML_reduced_imputed.csv')
X_train, X_test, y_train, y_test = split_data(df, split_data = 'split', outcome = 'commtot', predictor_columns=PREDICTOR_COLUMNS, test_size = 0.3, SEED=SEED)

model.fit(X_train, y_train)
# Create a SHAP explainer
explainer = shap.Explainer(model, X_train)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, show=False)

# save the shap figure
plt.savefig(figures_dir + f'{final_best_model_name}_shap_summary_plot.png', dpi=300, bbox_inches='tight')
plt.show()

# Conclusion:

The current study used machine learning models to examine how well PROs predicts severity of opioid misuse. 
First, we used an array of machine learning models to predict severity of opioid misuse ("commtot") with patient-reported outcome data. Elastic Net had shown the best performance. After a 100-fold marte-carlo cross validation, the mean correlation score was 0.61, RMSE = 4.1. 

Second, we performed an ablation analysis, where each feature is removed from the dataset, and then tested how much the removal impacts the model predictions. Among the predictors, Anger had the most reduction in prediction. Self-efficacy ("aeqtot") and pain catastrophizing ("pcs") had the second and third most reduction in prediction. All three highly impactful features are related to emotion processes, suggesting a viable psychological mechanisms underlying opioid misuse in pain. Moreover, it suggests psychological management and emotion regulation could be possible effective treatment pathways for opioid misuse, and future studies can test such hypothesis with clinical trials to test treatment efficacy. 